# <b>월간 데이콘 2 천체 유형 분류

##### 작성자 : 허은정
##### 마지막 수정 : 20.02.28 23:30

### <b>라이브러리 및 데이터

In [0]:
# 비활성화방지, F12 개발자도구. 콘솔창에 입력

function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)

#### 구글 드라이브에 마운트

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

#### 라이브러리 설치

In [0]:
import os                                           # 디렉토리 설정
os.chdir("/content/gdrive/My Drive/stella")
import warnings                                      # 경고 메세지 무시
warnings.filterwarnings('ignore')
import pandas as pd                                  # 데이터 조작, 분석
import numpy as np                                   # 행렬 연산
import random                                        # 난수 생성
random.seed(2020)
import time
from sklearn.preprocessing import OneHotEncoder      # OneHot인코딩

!pip install catboost
from catboost import CatBoostClassifier, Pool        # CatBoost 모델링
from lightgbm.sklearn import LGBMClassifier          # lightGBM 모델링
from sklearn.metrics import log_loss                 # LogLoss 측정

import matplotlib.pyplot as plt                      # 시각화

### <b>EDA

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)

###### 왼쪽 train 데이터, 오른쪽 test 데이터 분포 비교

In [0]:
plt.subplots(figsize=(20,40))
for i in range(20):
    plt.subplot(20, 2, 2*i+1)
    plt.title(train.keys()[3+i])
    plt.boxplot(train.iloc[:,3+i], vert=False)
for i in range(20):
    plt.subplot(20, 2, 2*i+2)
    plt.title(test.keys()[2+i])
    plt.boxplot(test.iloc[:,2+i], vert=False)
plt.show()

###### Groupby로 분포 파악

In [0]:
plt.subplots(figsize=(20,40))
for i in range(19):
    plt.subplot(19,1,i+1)
    plt.title(i+1)
    plt.plot(train.groupby(['type_num']).mean().iloc[i,3:], color=(i/19,0,0))
plt.show()

###### type 별 그래프

In [0]:
plt.subplots(figsize=(20,10))
for i in range(len(train[(train['type_num'] == 6)].iloc[10000:20000])):
  plt.plot(train.keys()[3:23],train[(train['type_num'] == 6)].iloc[10000:20000].iloc[i,3:23])

In [0]:
plt.subplots(figsize=(20,10))
for i in range(len(train[(train['type_num'] == 6)].iloc[10000:20000])):
  plt.plot(train.keys()[23:39],train[(train['type_num'] == 6)].iloc[10000:20000].iloc[i,23:39])

### <b> 모델 생성, 적용

###### catboost depth=6

In [0]:
def modeling(split_size, order, random_seed):

# 데이터 불러오기

  train = pd.read_csv("train.csv")
  test = pd.read_csv("test.csv")
  sample_submission = pd.read_csv('sample_submission.csv', index_col=0)
  

  # train 데이터 -> train, valid, last_valid로 나누기

  valid_index = len(train) // split_size
  valid = train.sample(frac=1, random_state=random_seed).iloc[(valid_index * (order-1)):(valid_index * order),:].reset_index(drop=True)
  train = pd.concat([train.sample(frac=1, random_state=random_seed).iloc[:(valid_index * (order-1)),:],
                     train.sample(frac=1, random_state=random_seed).iloc[(valid_index * order):,:]]).reset_index(drop=True)


  # test 데이터 범위에 train, valid, last_valid 데이터 값 맞추기
  # test 데이터 범위 ±1 밖에 있으면 Outlier로 취급, 제거

  for var in train.keys()[3:23]:
    test = test[(test[var] > 10) & (test[var] < 40)]

  for var in train.keys()[3:23]:
    train = train[(train[var] > test[var].min() - 1) & (train[var] < test[var].max() + 1)].reset_index(drop=True)
    valid = valid[(valid[var] > test[var].min() - 1) & (valid[var] < test[var].max() + 1)].reset_index(drop=True)

  test = pd.read_csv("test.csv")

  # psfMag_u ~ modelMag_z까지 20개 변수 각각의 차 계산, 20C2개 변수 추가

  for i in range(19):
    for j in range(19-i):
      train[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(train.iloc[:,3+i])-np.array(train.iloc[:,3+i+j+1]))
      valid[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(valid.iloc[:,3+i])-np.array(valid.iloc[:,3+i+j+1]))
      test[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(test.iloc[:,2+i])-np.array(test.iloc[:,2+i+j+1]))
  

  # fiberID별 psfMag_u ~ modelMag_z 20개 변수 각각의 평균값 계산, 20개 변수 추가

  group_mean = train.groupby(train['fiberID']).mean().iloc[:,2:22]
  train = pd.merge(train, group_mean, how='left', left_on='fiberID', right_index=True)
  valid = pd.merge(valid, group_mean, how='left', left_on='fiberID', right_index=True)   # valid, test에도 train값 merge
  test = pd.merge(test, group_mean, how='left', left_on='fiberID', right_index=True)


  # type을 sample_submission에 대응하는 가변수 형태로 변환

  column_number = {}
  for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
  def to_number(x, dic):
    return dic[x]
  train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))
  valid['type_num'] = valid['type'].apply(lambda x: to_number(x, column_number))


  # 독립변수 x, 종속변수 y로 분리

  train_x = train.drop(columns=['type', 'type_num', 'id', 'fiberID'], axis=1)
  train_y = train['type_num']
  valid_x = valid.drop(columns=['type', 'type_num', 'id', 'fiberID'], axis=1)
  valid_y = valid['type_num']
  test_x = test.drop(columns=['id', 'fiberID'], axis=1)


  # Catboost 모델 생성, 적용, last_valid_pred, test_pred 생성
  
  result = (random_seed-2020)*3+order

  model = CatBoostClassifier(iterations=50000,               # 반복횟수 최대 50000
                              learning_rate=0.05,           # 러닝레이트 0.05
                              depth=6,                       # 트리 깊이 ~16, 디폴트 6
                              l2_leaf_reg=30,                # L2 정규화
                              random_seed=random_seed,       # 랜덤시드 고정
                              metric_period=50000,           # 중간결과 출력안함
                              early_stopping_rounds=1000,    # 1000iteration 동안 val_loss 감소 없으면 학습 중단
                              grow_policy='SymmetricTree',   # 트리 생성 방식, SymmetricTree는 wide, 깊이 일정, Lossguide는 deep
                              task_type='GPU'                # GPU 사용
                             )

  model.fit(train_x, train_y, eval_set=(valid_x, valid_y))
  test_pred = model.predict_proba(test_x)


  # test_pred 값을 csv파일로 저장

  submission = pd.DataFrame(data=test_pred, columns=sample_submission.columns, index=sample_submission.index)
  submission.to_csv('t'+str(result)+'.csv', index=True)

  return str(result)+'번째 모델 생성 완료'
  

In [0]:
for j in range(5):
  for i in range(3):
    modeling(3,i+1,2020+j)

###### catboost depth=8

In [0]:
def modeling(split_size, order, random_seed):

# 데이터 불러오기

  train = pd.read_csv("train.csv")
  test = pd.read_csv("test.csv")
  sample_submission = pd.read_csv('sample_submission.csv', index_col=0)
  

  # train 데이터 -> train, valid, last_valid로 나누기

  valid_index = len(train) // split_size
  valid = train.sample(frac=1, random_state=random_seed).iloc[(valid_index * (order-1)):(valid_index * order),:].reset_index(drop=True)
  train = pd.concat([train.sample(frac=1, random_state=random_seed).iloc[:(valid_index * (order-1)),:],
                     train.sample(frac=1, random_state=random_seed).iloc[(valid_index * order):,:]]).reset_index(drop=True)


  # test 데이터 범위에 train, valid, last_valid 데이터 값 맞추기
  # test 데이터 범위 ±1 밖에 있으면 Outlier로 취급, 제거

  for var in train.keys()[3:23]:
    test = test[(test[var] > 10) & (test[var] < 40)]

  for var in train.keys()[3:23]:
    train = train[(train[var] > test[var].min() - 1) & (train[var] < test[var].max() + 1)].reset_index(drop=True)
    valid = valid[(valid[var] > test[var].min() - 1) & (valid[var] < test[var].max() + 1)].reset_index(drop=True)

  test = pd.read_csv("test.csv")

  # psfMag_u ~ modelMag_z까지 20개 변수 각각의 차 계산, 20C2개 변수 추가

  for i in range(19):
    for j in range(19-i):
      train[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(train.iloc[:,3+i])-np.array(train.iloc[:,3+i+j+1]))
      valid[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(valid.iloc[:,3+i])-np.array(valid.iloc[:,3+i+j+1]))
      test[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(test.iloc[:,2+i])-np.array(test.iloc[:,2+i+j+1]))
  

  # fiberID별 psfMag_u ~ modelMag_z 20개 변수 각각의 평균값 계산, 20개 변수 추가

  group_mean = train.groupby(train['fiberID']).mean().iloc[:,2:22]
  train = pd.merge(train, group_mean, how='left', left_on='fiberID', right_index=True)
  valid = pd.merge(valid, group_mean, how='left', left_on='fiberID', right_index=True)   # valid, test에도 train값 merge
  test = pd.merge(test, group_mean, how='left', left_on='fiberID', right_index=True)


  # type을 sample_submission에 대응하는 가변수 형태로 변환

  column_number = {}
  for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
  def to_number(x, dic):
    return dic[x]
  train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))
  valid['type_num'] = valid['type'].apply(lambda x: to_number(x, column_number))


  # 독립변수 x, 종속변수 y로 분리

  train_x = train.drop(columns=['type', 'type_num', 'id', 'fiberID'], axis=1)
  train_y = train['type_num']
  valid_x = valid.drop(columns=['type', 'type_num', 'id', 'fiberID'], axis=1)
  valid_y = valid['type_num']
  test_x = test.drop(columns=['id', 'fiberID'], axis=1)


  # Catboost 모델 생성, 적용, last_valid_pred, test_pred 생성
  
  result = 15 + (random_seed-2020)*3+order

  model = CatBoostClassifier(iterations=50000,               # 반복횟수 최대 50000
                              learning_rate=0.05,           # 러닝레이트 0.05
                              depth=8,                       # 트리 깊이 ~16, 디폴트 6
                              l2_leaf_reg=30,                # L2 정규화
                              random_seed=random_seed,       # 랜덤시드 고정
                              metric_period=50000,           # 중간결과 출력안함
                              early_stopping_rounds=1000,    # 1000iteration 동안 val_loss 감소 없으면 학습 중단
                              grow_policy='SymmetricTree',   # 트리 생성 방식, SymmetricTree는 wide, 깊이 일정, Lossguide는 deep
                              task_type='GPU'                # GPU 사용
                             )

  model.fit(train_x, train_y, eval_set=(valid_x, valid_y))
  test_pred = model.predict_proba(test_x)


  # test_pred 값을 csv파일로 저장

  submission = pd.DataFrame(data=test_pred, columns=sample_submission.columns, index=sample_submission.index)
  submission.to_csv('t'+str(result)+'.csv', index=True)

  return str(result)+'번째 모델 생성 완료'
  

In [0]:
for j in range(5):
  for i in range(3):
    modeling(3,i+1,2020+j)

###### lightgbm

In [0]:
def modeling(split_size, order, random_seed):

# 데이터 불러오기

  train = pd.read_csv("train.csv")
  test = pd.read_csv("test.csv")
  sample_submission = pd.read_csv('sample_submission.csv', index_col=0)
  

  # train 데이터 -> train, valid, last_valid로 나누기

  valid_index = len(train) // split_size
  valid = train.sample(frac=1, random_state=random_seed).iloc[(valid_index * (order-1)):(valid_index * order),:].reset_index(drop=True)
  train = pd.concat([train.sample(frac=1, random_state=random_seed).iloc[:(valid_index * (order-1)),:],
                     train.sample(frac=1, random_state=random_seed).iloc[(valid_index * order):,:]]).reset_index(drop=True)


  # test 데이터 범위에 train, valid, last_valid 데이터 값 맞추기
  # test 데이터 범위 ±1 밖에 있으면 Outlier로 취급, 제거

  for var in train.keys()[3:23]:
    test = test[(test[var] > 10) & (test[var] < 40)]

  for var in train.keys()[3:23]:
    train = train[(train[var] > test[var].min() - 1) & (train[var] < test[var].max() + 1)].reset_index(drop=True)
    valid = valid[(valid[var] > test[var].min() - 1) & (valid[var] < test[var].max() + 1)].reset_index(drop=True)

  test = pd.read_csv("test.csv")

  # psfMag_u ~ modelMag_z까지 20개 변수 각각의 차 계산, 20C2개 변수 추가

  for i in range(19):
    for j in range(19-i):
      train[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(train.iloc[:,3+i])-np.array(train.iloc[:,3+i+j+1]))
      valid[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(valid.iloc[:,3+i])-np.array(valid.iloc[:,3+i+j+1]))
      test[str(train.keys()[3+i])+'_'+str(train.keys()[3+i+j+1])] = pd.DataFrame(np.array(test.iloc[:,2+i])-np.array(test.iloc[:,2+i+j+1]))
  

  # fiberID별 psfMag_u ~ modelMag_z 20개 변수 각각의 평균값 계산, 20개 변수 추가

  group_mean = train.groupby(train['fiberID']).mean().iloc[:,2:22]
  train = pd.merge(train, group_mean, how='left', left_on='fiberID', right_index=True)
  valid = pd.merge(valid, group_mean, how='left', left_on='fiberID', right_index=True)   # valid, test에도 train값 merge
  test = pd.merge(test, group_mean, how='left', left_on='fiberID', right_index=True)


  # type을 sample_submission에 대응하는 가변수 형태로 변환

  column_number = {}
  for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
  def to_number(x, dic):
    return dic[x]
  train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))
  valid['type_num'] = valid['type'].apply(lambda x: to_number(x, column_number))


  # 독립변수 x, 종속변수 y로 분리

  train_x = train.drop(columns=['type', 'type_num', 'id', 'fiberID'], axis=1)
  train_y = train['type_num']
  valid_x = valid.drop(columns=['type', 'type_num', 'id', 'fiberID'], axis=1)
  valid_y = valid['type_num']
  test_x = test.drop(columns=['id', 'fiberID'], axis=1)


  # Catboost 모델 생성, 적용, last_valid_pred, test_pred 생성
  
  result = 30 + (random_seed-2020)*3+order

  model = LGBMClassifier()
  model.set_params(boosting_type='gbdt',
                   bagging_freq=10,
                   bagging_seed=2020,
                   bagging_fraction=0.7,
                   feature_fraction=0.7,
                   num_leaves=100, 
                   max_depth=-1, 
                   learning_rate=0.01, 
                   n_estimators=2500,
                   random_state=2020, 
                   n_jobs=-1,
                   importance_type='split',
                   is_training_metric=True,
                   eval_metric='logloss',
                   early_stopping_rounds=10,
                   )
  
  start = time.time()
  model.fit(train_x, train_y, eval_set=(valid_x, valid_y))
  print("time :", time.time() - start)

  test_pred = model.predict_proba(test_x)


  # test_pred 값을 csv파일로 저장

  submission = pd.DataFrame(data=test_pred, columns=sample_submission.columns, index=sample_submission.index)
  submission.to_csv('t'+str(result)+'.csv', index=True)

  return str(result)+'번째 모델 생성 완료'
  

In [0]:
for j in range(5):
  for i in range(3):
    modeling(3,i+1,2020+j)

### <b>모델 앙상블

In [0]:
n = 45
test_pred = pd.read_csv('t1.csv', index_col=0) /n
for i in range(n-1):
  test_pred += pd.read_csv('t'+str(i+2)+'.csv', index_col=0) /n

### <b>제출 파일 생성

In [0]:
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)
submission = pd.DataFrame(data=test_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('3mix_3fold_5.csv', index=True)